In [0]:
-- ============================================
-- Databricks SQL Analysis for Railway Data
-- Compatible with Databricks SQL & Delta Lake
-- ============================================

-- Created temporary view for analysis
CREATE OR REPLACE TEMPORARY VIEW railway_data AS
SELECT *
FROM workspace.default.railway;

-- ============================================
-- SECTION 1: OVERVIEW METRICS
-- ============================================

-- Total bookings and revenue

SELECT 
    COUNT(*) AS total_bookings,
    SUM(Price) AS total_revenue,
    AVG(Price) AS avg_ticket_price,
    MIN(Price) AS min_price,
    MAX(Price) AS max_price
FROM railway_data;
